In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/RGB_Li_27c_J3_nf4_v1.0.2.pth

In [68]:
# import os
import io
import torch
# import matplotlib.pyplot as plt
import numpy as np
# from PIL import Image, ImageEnhance
# from IPython.display import display
from torchvision.transforms import ToPILImage, PILToTensor
from datasets import load_dataset
from walloc import walloc
# from walloc.walloc import latent_to_pil, pil_to_latent
class Config: pass

In [64]:
device = "cuda:3"
checkpoint = torch.load("RGB_Li_27c_J3_nf4_v1.0.2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec2D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode
)
codec.load_state_dict(checkpoint['model_state_dict'])
codec = codec.to(device)
codec.eval();

In [37]:
dataset = load_dataset("danjacobellis/LSDIR", split='validation')

Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

In [71]:
sample = dataset[0]
img = sample['image']
with torch.no_grad():
    x = PILToTensor()(img).to(torch.float)
    x = (x/255 - 0.5).unsqueeze(0).to(device)
    x = walloc.pad(x,p=8)
    X = codec.wavelet_analysis(x,codec.J)
    Y = codec.encoder(X).to("cpu")
    webp = walloc.latent_to_pil(Y,codec.latent_bits, 3)[0]
    buff = io.BytesIO()
    webp.save(buff, format='WEBP', lossless=True)
    webp_bytes = buff.getbuffer()
    X_hat = codec.decoder(Y)
    # x_hat = codec.wavelet_synthesis(X_hat,codec.J)
    # mse = torch.nn.functional.mse_loss(x,x_hat).item()
    # rec = ToPILImage()(x_hat[0]+0.5)

In [72]:
len(webp_bytes)

367936